import modules

In [1]:
import cv2
import threading

import time
import numpy as np

import csv

# affine transform cal

* imshow 창(**cam1, cam2 각각의 창에서**)에서 탁구대 영역 총 4개 꼭짓점 클릭하기 - **우상, 좌상, 좌하, 우하 순으로 입력** 

In [2]:
# 마우스 이벤트 콜백 함수
coordinate_list = []
def mouse_click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("클릭됨 ", end="// ")
        coordinate_list.append([x, y])

# namedWindow 생성
cv2.namedWindow('Webcam') 
# 마우스 클릭 이벤트를 처리할 수 있도록 콜백 함수 등록
cv2.setMouseCallback('Webcam', mouse_click_event)

frame = cv2.imread('images/track-dataset/1cam1/1.jpg')  # cam1 탁구대 영역 변환 행렬 cal용 frame 경로 =============
print(f"<< cam1 start >>")
while True:
    # 읽은 프레임을 화면에 보여주기
    cv2.imshow('Webcam', frame)
    # 'q' 키를 누르면 루프에서 빠져나옴
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("")
        aprrox1 = coordinate_list
        coordinate_list = []
        break
assert len(aprrox1) == 4, "영역 꼭짓점은 4 점이어야 합니다. 다시 선택해주세요."
        
frame = cv2.imread('images/track-dataset/1cam2/1.jpg')  # cam2 탁구대 영역 변환 행렬 cal용 frame 경로 =============
print(f"<< cam2 start >>")
while True:
    # 읽은 프레임을 화면에 보여주기
    cv2.imshow('Webcam', frame)
    # 'q' 키를 누르면 루프에서 빠져나옴
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("")
        aprrox2 = coordinate_list
        coordinate_list = []
        break
assert len(aprrox2) == 4, "영역 꼭짓점은 4 점이어야 합니다. 다시 선택해주세요."

print("========================================")
print(f"aprrox1 = {aprrox1}")
print(f"aprrox2 = {aprrox2}")
        
# 모든 윈도우 닫기
cv2.destroyAllWindows()

<< cam1 start >>



AssertionError: 영역 꼭짓점은 4 점이어야 합니다. 다시 선택해주세요.

In [4]:
aprrox1 = [[533, 2], [52, 2], [49, 477], [547, 475]]
aprrox2 = [[593, 0], [106, 1], [109, 476], [612, 475]]

* 반환된 approx1, approx2를 이용해 **affine transform용 (matrix, w, h) 계산**

In [6]:
# def approx_to_affinePar
def approx_to_affinePar(approx, num, testImgPath):
    assert len(approx) == 4, "영역 꼭짓점은 4 점이어야 합니다. "
    # 우상, 좌상, 좌하, 우하 순으로 입력되었는지 확인 필요 -----------
    w = approx[3][0] - approx[2][0]
    h = max(approx[2][1] - approx[1][1], approx[3][1] - approx[0][1])
    # affine matrix 구하기
    pts1 = np.float32(approx)
    pts2 = np.float32([[w, 0], [0, 0], [0, h], [w, h]]) 
    matrix = cv2.getPerspectiveTransform(pts1, pts2) 
    # 출력 
    print(f"matrix{num} = np.array({matrix.tolist()})")
    print(f"w{num} = {w}")
    print(f"h{num} = {h}")
    # affine trans TEST 
    frame = cv2.imread(testImgPath)
    roi = cv2.warpPerspective(frame, matrix, (w, h))
    cv2.imshow(f'frame{num}', frame)
    if roi is not None:
        cv2.imshow(f'ROI{num}', roi)
    
# 실행 
approx_to_affinePar(aprrox1, 1, 'images/track-dataset/1cam1/1.jpg')
approx_to_affinePar(aprrox2, 2, 'images/track-dataset/1cam2/1.jpg')

cv2.waitKey(0)
cv2.destroyAllWindows()

matrix1 = np.array([[1.030833006743526, 0.006510524253116999, -53.61633739916959], [-1.688675606011207e-16, 1.0351654514342388, -2.0703309028684735], [-8.452631594944291e-06, 7.459042008886985e-05, 1.0]])
w1 = 498
h1 = 475
matrix2 = np.array([[1.033012029144146, -0.006524286499857764, -109.49275080277963], [0.002121027020147488, 1.0329401588118488, -1.2577690229474694], [2.7954525695380265e-07, 6.916614356926089e-05, 1.0]])
w2 = 503
h2 = 475


* 출력된 matrix, w, h 를 아래에 복붙해서 실행 ㄱㄱ - 변수 설정용 

In [2]:
matrix1 = np.array([[1.030833006743526, 0.006510524253116999, -53.61633739916959], [-1.688675606011207e-16, 1.0351654514342388, -2.0703309028684735], [-8.452631594944291e-06, 7.459042008886985e-05, 1.0]])
w1 = 498
h1 = 475
matrix2 = np.array([[1.033012029144146, -0.006524286499857764, -109.49275080277963], [0.002121027020147488, 1.0329401588118488, -1.2577690229474694], [2.7954525695380265e-07, 6.916614356926089e-05, 1.0]])
w2 = 503
h2 = 475

# ==================================================

# data collecting - 수동 구간 지정 TEST

In [3]:
class CameraThread(threading.Thread):
    def __init__(self, camID):
        threading.Thread.__init__(self)
        self.camID = camID
        self.cap = cv2.VideoCapture(self.camID)
        self.current_frame = None
        self.running = True  # Control the running state of the thread

    def run(self):
        while self.running:
            ret, self.current_frame = self.cap.read()

    def close(self):
        self.running = False  # Signal the thread to stop
        self.join()  # Wait for the thread to finish
        self.cap.release()

In [4]:
def orange_in_tableROI(frame, num):
    # RGB to HSV 변환
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # 주황색 범위 설정 (HSV 값)
    lower_orange = np.array([0, 50, 50])
    upper_orange = np.array([30, 255, 255])
    # 주황색 객체를 분리하기 위한 마스크 생성
    orange_mask = cv2.inRange(hsv, lower_orange, upper_orange)
    # 마스크를 이용하여 원본 이미지에서 주황색 객체만 추출
    orange_object = cv2.bitwise_and(frame, frame, mask=orange_mask) 
    
    cv2.imshow(f'ROI{num} - orange', orange_object)

In [5]:
def ballDetect_in_tableROI(frame, num):
    # RGB to HSV 변환
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # 주황색 범위 설정 (HSV 값)
    lower_orange = np.array([0, 50, 50])
    upper_orange = np.array([30, 255, 255])
    # 주황색 객체를 분리하기 위한 마스크 생성
    orange_mask = cv2.inRange(hsv, lower_orange, upper_orange)
    # 마스크를 이용하여 원본 이미지에서 주황색 객체만 추출
    orange_object = cv2.bitwise_and(frame, frame, mask=orange_mask) 

    ########## 탁구공 영역 중심 좌표 구하기 ############################################

    # find contours in the binary image
    contours, hierarchy = cv2.findContours(orange_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # 초기 면적을 0으로 설정합니다.
    max_area = 0
    largest_contour = None
    # 모든 contour에 대해 반복합니다.
    if len(contours) == 1: 
        largest_contour = contours[0]
    else: 
        for c in contours:
            # 각 contour의 면적을 계산합니다.
            area = cv2.contourArea(c)
            # 현재 contour의 면적이 이전 최대 면적보다 크다면,
            # 이 contour를 가장 큰 contour로 간주하고 면적을 갱신합니다.
            if area > max_area:
                max_area = area
                largest_contour = c
            
    # 중심점 좌표 계산 
    if largest_contour is not None:
        # calculate moments for the largest contour
        M = cv2.moments(largest_contour)
        # calculate x,y coordinate of center
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX, cY = 0, 0
        # print coordinates
        print(f"cam{num} ----- ({cX}, {cY})") # ball (x, y) is 
        # draw a circle around the center point
        cv2.circle(orange_object, (cX, cY), 3, (255, 0, 0), -1)
                   
        cv2.imshow(f'ROI{num} - ball', orange_object)
        
    ##########################################################################
    

In [6]:
cam0 = CameraThread(1)
cam1 = CameraThread(2)
cam0.start()
cam1.start()

frame_counter0 = 0
frame_counter1 = 0
save_frames = False
ball_start = False

In [7]:
while True:
    frame0 = cam0.current_frame
    frame1 = cam1.current_frame

    if frame0 is not None:
        cv2.imshow('Camera 0', frame0)
        ROI0 = cv2.warpPerspective(frame0, matrix1, (w1, h1))
        cv2.imshow('ROI0', ROI0)
        orange_in_tableROI(ROI0, 1)
        if ball_start: 
            ballDetect_in_tableROI(ROI0, 1)

    if frame1 is not None:
        cv2.imshow('Camera 1', frame1)
        ROI1 = cv2.warpPerspective(frame1, matrix2, (w2, h2))
        cv2.imshow('ROI1', ROI1)
        orange_in_tableROI(ROI1, 2) 
        if ball_start: 
            ballDetect_in_tableROI(ROI1, 2)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('i'):
        print("출발-----------")
        ball_start = True
    elif key == ord('o'):
        print("도착-----------")
        ball_start = False

In [ ]:
cam0.join()
cam1.join()
cam0.close()  
cam1.close()

cv2.destroyAllWindows()

In [9]:
raw_data = """
cam1 ----- (0, 0)
cam2 ----- (431, 4)
cam1 ----- (0, 0)
cam2 ----- (431, 4)
cam1 ----- (348, 17)
cam2 ----- (431, 4)
cam1 ----- (348, 17)
cam2 ----- (418, 90)
cam1 ----- (336, 110)
cam2 ----- (418, 90)
cam1 ----- (336, 110)
cam2 ----- (418, 90)
cam1 ----- (336, 110)
cam2 ----- (400, 198)
cam1 ----- (322, 208)
cam2 ----- (400, 198)
cam1 ----- (322, 208)
cam2 ----- (379, 297)
cam1 ----- (322, 208)
cam2 ----- (379, 297)
cam1 ----- (322, 208)
cam2 ----- (379, 297)
cam1 ----- (322, 208)
cam2 ----- (360, 382)
cam1 ----- (322, 208)
cam2 ----- (360, 382)
cam1 ----- (301, 374)
cam2 ----- (360, 382)
cam1 ----- (301, 374)
cam2 ----- (360, 382)
cam1 ----- (301, 374)
cam2 ----- (342, 449)
cam1 ----- (292, 438)
cam2 ----- (342, 449)
cam1 ----- (292, 438)
cam2 ----- (342, 449)
"""

cam1_points = []
cam2_points = []

for line in raw_data.strip().split('\n'):
    cam, coords = line.split("-----")
    x, y = eval(coords.strip())
    if "cam1" in cam:
        cam1_points.append((x, y))
    elif "cam2" in cam:
        cam2_points.append((x, y))

print("cam1_points = ", cam1_points)
print("cam2_points = ", cam2_points)

cam1_points =  [(0, 0), (0, 0), (348, 17), (348, 17), (336, 110), (336, 110), (336, 110), (322, 208), (322, 208), (322, 208), (322, 208), (322, 208), (322, 208), (301, 374), (301, 374), (301, 374), (292, 438), (292, 438)]
cam2_points =  [(431, 4), (431, 4), (431, 4), (418, 90), (418, 90), (418, 90), (400, 198), (400, 198), (379, 297), (379, 297), (379, 297), (360, 382), (360, 382), (360, 382), (360, 382), (342, 449), (342, 449), (342, 449)]


In [13]:
frame1 = cv2.imread('images/track-dataset/1cam1/1.jpg')
frame2 = cv2.imread('images/track-dataset/1cam2/1.jpg')

frame1 = cv2.warpPerspective(frame1, matrix1, (w1, h1))
frame2 = cv2.warpPerspective(frame2, matrix2, (w2, h2))

cam1_points =  [(0, 0), (0, 0), (348, 17), (348, 17), (336, 110), (336, 110), (336, 110), (322, 208), (322, 208), (322, 208), (322, 208), (322, 208), (322, 208), (301, 374), (301, 374), (301, 374), (292, 438), (292, 438)]
cam2_points =  [(431, 4), (431, 4), (431, 4), (418, 90), (418, 90), (418, 90), (400, 198), (400, 198), (379, 297), (379, 297), (379, 297), (360, 382), (360, 382), (360, 382), (360, 382), (342, 449), (342, 449), (342, 449)]

for point in cam1_points:
    cv2.circle(frame1, point, 5, (0, 255, 0), -1)
for point in cam2_points:
    cv2.circle(frame2, point, 5, (0, 255, 0), -1)

cv2.imshow('Camera 1 - Ball Points', frame1)
cv2.imshow('Camera 2 - Ball Points', frame2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
with open('dataset_ballPosXY.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(['cam1']+[str(x) for x in cam1_points]) 
    writer.writerow(['cam2']+[str(x) for x in cam2_points]) 
    
    writer.writerow(['cam1']+[str(x) for x in cam1_points]) 
    writer.writerow(['cam2']+[str(x) for x in cam2_points]) 

# ==================================================

In [27]:
with open('dataset_ballPosXY.csv', 'w', newline='') as file:
    writer = csv.writer(file)

# data collecting - 자동 구간 지정 TEST

In [3]:
cap1 = cv2.VideoCapture(1)
cap2 = cv2.VideoCapture(2)

In [4]:
class CameraThread(threading.Thread):
    def __init__(self, cap):
        threading.Thread.__init__(self)
#         self.camID = camID
        self.cap = cap # cv2.VideoCapture(self.camID)
        self.current_frame = None
        self.running = True  # Control the running state of the thread

    def run(self):
        while self.running:
            ret, self.current_frame = self.cap.read()

    def close(self):
        self.running = False  # Signal the thread to stop
        # self.join()  # Wait for the thread to finish
        # self.cap.release()

In [5]:
# state 변수 
frame_counter0 = 0
frame_counter1 = 0
dataColumn_counter0 = 0
dataColumn_counter1 = 0
save_frames = False
is_orange_in_range = False

In [6]:
def orange_in_tableROI(frame, num):
    # RGB to HSV 변환
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # 주황색 범위 설정 (HSV 값)
    lower_orange = np.array([0, 50, 50])
    upper_orange = np.array([30, 255, 255])
    # 주황색 객체를 분리하기 위한 마스크 생성
    orange_mask = cv2.inRange(hsv, lower_orange, upper_orange)
    # 마스크를 이용하여 원본 이미지에서 주황색 객체만 추출
    orange_object = cv2.bitwise_and(frame, frame, mask=orange_mask) 
    # 반환 
#     cv2.imshow(f'ROI{num} - orange', orange_object)
    return orange_mask, orange_object

In [7]:
def check_orange_in_range(orange_object, ymin, ymax):
    # 주황색 픽셀의 좌표를 찾습니다. 
    orange_pixels = cv2.findNonZero(orange_object)
    # 픽셀 좌표가 없으면, 주황색 물체가 없는 것이므로 False를 반환합니다.
    if orange_pixels is None:
        return False
    # 픽셀 좌표가 범위 내에 있으면, True를 반환합니다. 
    for pixel in orange_pixels:
        y = pixel[0][1]  # y 좌표를 얻습니다.
        if ymin <= y <= ymax:  # 픽셀의 y 좌표가 지정한 범위 내에 있는지 확인합니다.
            return True
    # 지정한 범위 내에 주황색 픽셀이 없으면 False를 반환합니다.
    return False 

In [8]:
def ballPosition_in_tableROI(frame, orange_mask, orange_object, num):
    # find contours in the binary image
    contours, hierarchy = cv2.findContours(orange_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)    
    # 초기 면적을 0으로 설정합니다.
        max_area = 0
        largest_contour = None 
    # 모든 contour에 대해 반복합니다.
    if len(contours) == 1: 
        largest_contour = contours[0]
    else: 
        for c in contours:
            # 각 contour의 면적을 계산합니다.
            area = cv2.contourArea(c)
            # 현재 contour의 면적이 이전 최대 면적보다 크다면,
            # 이 contour를 가장 큰 contour로 간주하고 면적을 갱신합니다.
            if area > max_area:
                max_area = area
                largest_contour = c
    # 중심점 좌표 계산 
    if largest_contour is not None:
        # calculate moments for the largest contour
        M = cv2.moments(largest_contour)
        # calculate x,y coordinate of center
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX, cY = 0, 0
        # print coordinates
        print(f"cam{num} ----- ({cX}, {cY})") # ball (x, y) is 
        # draw a circle around the center point 
#         cv2.circle(orange_object, (cX, cY), 3, (255, 0, 0), -1)
#         cv2.imshow(f'ROI{num} - ball', orange_object)
        return str((cX, cY))

In [10]:
cam0 = CameraThread(cap1)
cam1 = CameraThread(cap2)
# cam0 = CameraThread(1)
# cam1 = CameraThread(2)
cam0.start()
cam1.start()

with open('dataset_ballPosXY.csv', 'w', newline='') as file:
    writer = csv.writer(file)
cam1_points = []
cam2_points = []

In [11]:
while True:
    frame0 = cam0.current_frame
    frame1 = cam1.current_frame

    if frame0 is not None or frame1 is not None:
        # cv2.imshow('Camera 0', frame0)
        # cv2.imshow('Camera 1', frame1)
        ROI0 = cv2.warpPerspective(frame0, matrix1, (w1, h1))
        ROI1 = cv2.warpPerspective(frame1, matrix2, (w2, h2))
#         cv2.imshow('ROI0', ROI0)
#         cv2.imshow('ROI1', ROI1)
        orange_mask0, orange_object0 = orange_in_tableROI(ROI0, 1)
        orange_mask1, orange_object1 = orange_in_tableROI(ROI1, 2)
        cv2.imshow(f'ROI0 - orange', orange_object0)
        cv2.imshow(f'ROI1 - orange', orange_object1)
        is_orange_in_range = check_orange_in_range(orange_mask0, 0, 160)
        # is_orange_in_range = check_orange_in_range(orange_mask1, 0, 160)
        if is_orange_in_range: 
            save_frames = True
        if save_frames:
            ballPos0 = ballPosition_in_tableROI(ROI0, orange_mask0, orange_object0, 1)
            ballPos1 = ballPosition_in_tableROI(ROI1, orange_mask1, orange_object1, 2)
            cam1_points.append(ballPos0)
            cam2_points.append(ballPos1)
#             cv2.imwrite(f'images/ballPosDataCollect/col{dataColumn_counter0}_cam1_{frame_counter0}.jpg', ROI0)
#             cv2.imwrite(f'images/ballPosDataCollect/col{dataColumn_counter1}_cam2_{frame_counter1}.jpg', ROI1)
#             print(f'col{dataColumn_counter0}_cam1_{frame_counter0}.jpg saved')
#             print(f'col{dataColumn_counter1}_cam2_{frame_counter1}.jpg saved')
            frame_counter0 += 1 
            frame_counter1 += 1 
            if frame_counter0 >= 8: 
                with open('dataset_ballPosXY.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['cam1']+[str(x) for x in cam1_points]) 
                save_frames = False 
                frame_counter0 = 0 
                cam1_points = []
                dataColumn_counter0 += 1 
            if frame_counter1 >= 8: 
                with open('dataset_ballPosXY.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['cam2']+[str(x) for x in cam2_points]) 
                save_frames = False 
                frame_counter1 = 0 
                cam2_points = []
                dataColumn_counter1 += 1 

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break

# cam0.join()
# cam1.join()
cam0.close()  
cam1.close()

cv2.destroyAllWindows()

cam1 ----- (200, 39)
cam1 ----- (200, 39)
cam1 ----- (200, 39)
cam1 ----- (200, 39)
cam1 ----- (200, 39)
cam1 ----- (187, 114)
cam1 ----- (187, 114)
cam1 ----- (187, 114)
cam1 ----- (187, 114)
cam1 ----- (187, 114)
cam1 ----- (187, 114)
cam1 ----- (180, 184)
cam1 ----- (180, 184)
cam1 ----- (180, 184)
cam1 ----- (180, 184)
cam1 ----- (178, 242)
cam1 ----- (214, 17)
cam1 ----- (214, 17)
cam1 ----- (214, 17)
cam1 ----- (214, 17)
cam1 ----- (214, 17)
cam1 ----- (214, 17)
cam1 ----- (219, 158)
cam1 ----- (219, 158)
cam1 ----- (219, 158)
cam1 ----- (219, 158)
cam1 ----- (219, 158)
cam1 ----- (219, 158)
cam1 ----- (219, 158)
cam1 ----- (228, 293)
cam1 ----- (228, 293)
cam1 ----- (228, 293)
cam1 ----- (215, 123)
cam1 ----- (215, 123)
cam1 ----- (215, 123)
cam1 ----- (222, 289)
cam1 ----- (222, 289)
cam1 ----- (222, 289)
cam1 ----- (222, 289)
cam1 ----- (222, 289)
cam1 ----- (214, 11)
cam1 ----- (214, 11)
cam1 ----- (214, 11)
cam1 ----- (214, 11)
cam1 ----- (214, 11)
cam1 ----- (220, 177)
cam1

### =---------------------------------------

In [ ]:
import queue

In [ ]:
class CamWithProc():
    def __init__(self, camID):
        self.camID = camID 
        self.cap = cv2.VideoCapture(self.camID)
        self.current_frame = None

    def proc(self, message): 
        if 
        
        
    def close(self):
        self.cap.release() 

In [3]:
class CamProcessThread(threading.Thread):
    def __init__(self, camID):
        threading.Thread.__init__(self)
        self.cam = CamWithProc(camID)
        # self.cam.init()
        self.queue = queue.Queue() 
        
        self.running = True  # Control the running state of the thread
        
    def add_proc(self, action, message=None):
        self.queue.put((action, message))
        
    def run(self): 
        while self.running: 
            ret, self.cam.current_frame = self.cam.cap.read()
            action, message = self.queue.get()
            if action == "": 

    def stop(self):
        self.running = False  # Signal the thread to stop
        self.join()  # Wait for the thread to finish

In [17]:
while True:
    frame0 = cam0.current_frame
    frame1 = cam1.current_frame

    if frame0 is not None:
        # cv2.imshow('Camera 0', frame0)
        ROI0 = cv2.warpPerspective(frame0, matrix1, (w1, h1))
        cv2.imshow('ROI0', ROI0)
        orange_mask0, orange_object0 = orange_in_tableROI(ROI0, 1)
        cv2.imshow(f'ROI0 - orange', orange_object0)
        is_orange_in_range = check_orange_in_range(orange_mask0, 0, 160)
        if is_orange_in_range: 
            save_frames = True
        if save_frames:
            ballPos0 = ballPosition_in_tableROI(ROI0, orange_mask0, orange_object0, 1)
            cam1_points.append(ballPos0)
            cv2.imwrite(f'images/ballPosDataCollect/col{dataColumn_counter0}_cam1_{frame_counter0}.jpg', ROI0)
            print(f'col{dataColumn_counter0}_cam1_{frame_counter0}.jpg saved')
            frame_counter0 += 1 
            if frame_counter0 >= 8: 
                with open('dataset_ballPosXY.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['cam1']+[str(x) for x in cam1_points]) 
                save_frames = False 
                frame_counter0 = 0 
                cam1_points = []
                dataColumn_counter0 += 1 

    if frame1 is not None:
        # cv2.imshow('Camera 1', frame1)
        ROI1 = cv2.warpPerspective(frame1, matrix2, (w2, h2))
        cv2.imshow('ROI1', ROI1)
        orange_mask1, orange_object1 = orange_in_tableROI(ROI1, 2)
        cv2.imshow(f'ROI1 - orange', orange_object1)
        is_orange_in_range = check_orange_in_range(orange_mask1, 0, 160)
        if is_orange_in_range: 
            save_frames = True
        if save_frames:
            ballPos1 = ballPosition_in_tableROI(ROI1, orange_mask1, orange_object1, 2)
            cam2_points.append(ballPos1)
            cv2.imwrite(f'images/ballPosDataCollect/col{dataColumn_counter1}_cam2_{frame_counter1}.jpg', ROI1)
            print(f'col{dataColumn_counter1}_cam2_{frame_counter1}.jpg saved')
            frame_counter1 += 1 
            if frame_counter1 >= 8: 
                with open('dataset_ballPosXY.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['cam2']+[str(x) for x in cam2_points]) 
                save_frames = False 
                frame_counter1 = 0 
                cam2_points = []
                dataColumn_counter1 += 1 

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break

cam1 ----- (64, 20)
col0_cam1_0.jpg saved
cam2 ----- (72, 22)
col0_cam2_0.jpg saved
cam1 ----- (64, 20)
col0_cam1_1.jpg saved
cam2 ----- (72, 22)
col0_cam2_1.jpg saved
cam1 ----- (64, 20)
col0_cam1_2.jpg saved
cam2 ----- (72, 22)
col0_cam2_2.jpg saved
cam1 ----- (64, 20)
col0_cam1_3.jpg saved
cam2 ----- (72, 22)
col0_cam2_3.jpg saved
cam1 ----- (64, 20)
col0_cam1_4.jpg saved
cam2 ----- (72, 22)
col0_cam2_4.jpg saved
cam1 ----- (64, 20)
col0_cam1_5.jpg saved
cam2 ----- (72, 22)
col0_cam2_5.jpg saved
cam1 ----- (64, 20)
col0_cam1_6.jpg saved
cam2 ----- (72, 22)
col0_cam2_6.jpg saved
cam1 ----- (64, 20)
col0_cam1_7.jpg saved
cam2 ----- (72, 22)
col0_cam2_7.jpg saved
cam1 ----- (64, 20)
col1_cam1_0.jpg saved
cam2 ----- (72, 22)
col1_cam2_0.jpg saved
cam1 ----- (64, 20)
col1_cam1_1.jpg saved
cam2 ----- (72, 22)
col1_cam2_1.jpg saved
cam1 ----- (64, 20)
col1_cam1_2.jpg saved
cam2 ----- (72, 22)
col1_cam2_2.jpg saved
cam1 ----- (64, 20)
col1_cam1_3.jpg saved
cam2 ----- (72, 22)
col1_cam2_3.jp

col12_cam2_2.jpg saved
cam1 ----- (64, 20)
col12_cam1_3.jpg saved
cam2 ----- (72, 22)
col12_cam2_3.jpg saved
cam1 ----- (64, 20)
col12_cam1_4.jpg saved
cam2 ----- (72, 22)
col12_cam2_4.jpg saved
cam1 ----- (64, 20)
col12_cam1_5.jpg saved
cam2 ----- (72, 22)
col12_cam2_5.jpg saved
cam1 ----- (64, 20)
col12_cam1_6.jpg saved
cam2 ----- (72, 22)
col12_cam2_6.jpg saved
cam1 ----- (64, 20)
col12_cam1_7.jpg saved
cam2 ----- (72, 22)
col12_cam2_7.jpg saved
cam1 ----- (64, 20)
col13_cam1_0.jpg saved
cam2 ----- (72, 22)
col13_cam2_0.jpg saved
cam1 ----- (64, 20)
col13_cam1_1.jpg saved
cam2 ----- (72, 22)
col13_cam2_1.jpg saved
cam1 ----- (64, 20)
col13_cam1_2.jpg saved
cam2 ----- (72, 22)
col13_cam2_2.jpg saved
cam1 ----- (64, 20)
col13_cam1_3.jpg saved
cam2 ----- (72, 22)
col13_cam2_3.jpg saved
cam1 ----- (64, 20)
col13_cam1_4.jpg saved
cam2 ----- (72, 22)
col13_cam2_4.jpg saved
cam1 ----- (64, 20)
col13_cam1_5.jpg saved
cam2 ----- (72, 22)
col13_cam2_5.jpg saved
cam1 ----- (64, 20)
col13_cam1_

cam1 ----- (64, 20)
col24_cam1_5.jpg saved
cam2 ----- (72, 22)
col24_cam2_5.jpg saved
cam1 ----- (64, 20)
col24_cam1_6.jpg saved
cam2 ----- (72, 22)
col24_cam2_6.jpg saved
cam1 ----- (64, 20)
col24_cam1_7.jpg saved
cam2 ----- (72, 22)
col24_cam2_7.jpg saved
cam1 ----- (64, 20)
col25_cam1_0.jpg saved
cam2 ----- (72, 22)
col25_cam2_0.jpg saved
cam1 ----- (64, 20)
col25_cam1_1.jpg saved
cam2 ----- (72, 22)
col25_cam2_1.jpg saved
cam1 ----- (64, 20)
col25_cam1_2.jpg saved
cam2 ----- (72, 22)
col25_cam2_2.jpg saved
cam1 ----- (64, 20)
col25_cam1_3.jpg saved
cam2 ----- (72, 22)
col25_cam2_3.jpg saved
cam1 ----- (64, 20)
col25_cam1_4.jpg saved
cam2 ----- (72, 22)
col25_cam2_4.jpg saved
cam1 ----- (64, 20)
col25_cam1_5.jpg saved
cam2 ----- (72, 22)
col25_cam2_5.jpg saved
cam1 ----- (64, 20)
col25_cam1_6.jpg saved
cam2 ----- (72, 22)
col25_cam2_6.jpg saved
cam1 ----- (64, 20)
col25_cam1_7.jpg saved
cam2 ----- (72, 22)
col25_cam2_7.jpg saved
cam1 ----- (64, 20)
col26_cam1_0.jpg saved
cam2 ----- 

col36_cam2_7.jpg saved
cam1 ----- (64, 20)
col37_cam1_0.jpg saved
cam2 ----- (72, 22)
col37_cam2_0.jpg saved
cam1 ----- (64, 20)
col37_cam1_1.jpg saved
cam2 ----- (72, 22)
col37_cam2_1.jpg saved
cam1 ----- (64, 20)
col37_cam1_2.jpg saved
cam2 ----- (72, 22)
col37_cam2_2.jpg saved
cam1 ----- (64, 20)
col37_cam1_3.jpg saved
cam2 ----- (72, 22)
col37_cam2_3.jpg saved
cam1 ----- (64, 20)
col37_cam1_4.jpg saved
cam2 ----- (72, 22)
col37_cam2_4.jpg saved
cam1 ----- (64, 20)
col37_cam1_5.jpg saved
cam2 ----- (72, 22)
col37_cam2_5.jpg saved
cam1 ----- (64, 20)
col37_cam1_6.jpg saved
cam2 ----- (72, 22)
col37_cam2_6.jpg saved
cam1 ----- (64, 20)
col37_cam1_7.jpg saved
cam2 ----- (72, 22)
col37_cam2_7.jpg saved
cam1 ----- (64, 20)
col38_cam1_0.jpg saved
cam2 ----- (72, 22)
col38_cam2_0.jpg saved
cam1 ----- (64, 20)
col38_cam1_1.jpg saved
cam2 ----- (72, 22)
col38_cam2_1.jpg saved
cam1 ----- (64, 20)
col38_cam1_2.jpg saved
cam2 ----- (72, 22)
col38_cam2_2.jpg saved
cam1 ----- (64, 20)
col38_cam1_

cam1 ----- (64, 20)
col49_cam1_3.jpg saved
cam2 ----- (72, 22)
col49_cam2_3.jpg saved
cam1 ----- (64, 20)
col49_cam1_4.jpg saved
cam2 ----- (72, 22)
col49_cam2_4.jpg saved
cam1 ----- (64, 20)
col49_cam1_5.jpg saved
cam2 ----- (72, 22)
col49_cam2_5.jpg saved
cam1 ----- (64, 20)
col49_cam1_6.jpg saved
cam2 ----- (72, 22)
col49_cam2_6.jpg saved
cam1 ----- (64, 20)
col49_cam1_7.jpg saved
cam2 ----- (72, 22)
col49_cam2_7.jpg saved
cam1 ----- (64, 20)
col50_cam1_0.jpg saved
cam2 ----- (72, 22)
col50_cam2_0.jpg saved
cam1 ----- (64, 20)
col50_cam1_1.jpg saved
cam2 ----- (72, 22)
col50_cam2_1.jpg saved
cam1 ----- (64, 20)
col50_cam1_2.jpg saved
cam2 ----- (72, 22)
col50_cam2_2.jpg saved
cam1 ----- (64, 20)
col50_cam1_3.jpg saved
cam2 ----- (72, 22)
col50_cam2_3.jpg saved
cam1 ----- (64, 20)
col50_cam1_4.jpg saved
cam2 ----- (72, 22)
col50_cam2_4.jpg saved
cam1 ----- (64, 20)
col50_cam1_5.jpg saved
cam2 ----- (72, 22)
col50_cam2_5.jpg saved
cam1 ----- (64, 20)
col50_cam1_6.jpg saved
cam2 ----- 

...

col285_cam1_6.jpg saved
col272_cam2_4.jpg saved
cam1 ----- (393, 8)
col285_cam1_7.jpg saved
cam1 ----- (393, 8)
col286_cam1_0.jpg saved
col272_cam2_5.jpg saved
cam1 ----- (393, 8)
col286_cam1_1.jpg saved
col272_cam2_6.jpg saved
cam1 ----- (392, 7)
col286_cam1_2.jpg saved
col272_cam2_7.jpg saved
cam1 ----- (392, 7)
col286_cam1_3.jpg saved
col273_cam2_0.jpg saved
cam1 ----- (392, 8)
col286_cam1_4.jpg saved
col273_cam2_1.jpg saved
cam1 ----- (392, 8)
col286_cam1_5.jpg saved
col273_cam2_2.jpg saved
cam1 ----- (392, 8)
col286_cam1_6.jpg saved
col273_cam2_3.jpg saved
cam1 ----- (392, 8)
col286_cam1_7.jpg saved
cam1 ----- (392, 8)
col287_cam1_0.jpg saved
col273_cam2_4.jpg saved
cam1 ----- (393, 9)
col287_cam1_1.jpg saved
col273_cam2_5.jpg saved
cam1 ----- (393, 9)
col287_cam1_2.jpg saved
col273_cam2_6.jpg saved
cam1 ----- (393, 8)
col287_cam1_3.jpg saved
col273_cam2_7.jpg saved
cam1 ----- (392, 8)
col287_cam1_4.jpg saved
col274_cam2_0.jpg saved
cam1 ----- (392, 8)
col287_cam1_5.jpg saved
col2

col286_cam2_0.jpg saved
cam1 ----- (315, 348)
col301_cam1_2.jpg saved
cam2 ----- (339, 355)
col286_cam2_1.jpg saved
cam1 ----- (317, 334)
col301_cam1_3.jpg saved
cam2 ----- (345, 340)
col286_cam2_2.jpg saved
cam1 ----- (317, 334)
col301_cam1_4.jpg saved
cam2 ----- (350, 324)
col286_cam2_3.jpg saved
cam1 ----- (317, 334)
col301_cam1_5.jpg saved
cam2 ----- (354, 307)
col286_cam2_4.jpg saved
cam1 ----- (320, 303)
col301_cam1_6.jpg saved
cam2 ----- (354, 307)
col286_cam2_5.jpg saved
cam1 ----- (322, 286)
col301_cam1_7.jpg saved
cam1 ----- (323, 269)
col302_cam1_0.jpg saved
cam2 ----- (359, 271)
col286_cam2_6.jpg saved
cam1 ----- (324, 253)
col302_cam1_1.jpg saved
cam2 ----- (359, 253)
col286_cam2_7.jpg saved
cam1 ----- (325, 237)
col302_cam1_2.jpg saved
cam2 ----- (358, 235)
col287_cam2_0.jpg saved
cam1 ----- (327, 221)
col302_cam1_3.jpg saved
cam2 ----- (357, 220)
col287_cam2_1.jpg saved
cam1 ----- (327, 207)
col302_cam1_4.jpg saved
cam2 ----- (355, 205)
col287_cam2_2.jpg saved
cam1 -----

col298_cam2_5.jpg saved
col315_cam1_1.jpg saved
col298_cam2_6.jpg saved
col315_cam1_2.jpg saved
col298_cam2_7.jpg saved
cam1 ----- (392, 8)
col315_cam1_3.jpg saved
col299_cam2_0.jpg saved
cam1 ----- (392, 8)
col315_cam1_4.jpg saved
col299_cam2_1.jpg saved
cam1 ----- (393, 9)
col315_cam1_5.jpg saved
col299_cam2_2.jpg saved
cam1 ----- (393, 8)
col315_cam1_6.jpg saved
cam2 ----- (0, 0)
col299_cam2_3.jpg saved
cam1 ----- (392, 8)
col315_cam1_7.jpg saved
cam1 ----- (393, 9)
col316_cam1_0.jpg saved
col299_cam2_4.jpg saved
cam1 ----- (393, 9)
col316_cam1_1.jpg saved
col299_cam2_5.jpg saved
cam1 ----- (392, 7)
col316_cam1_2.jpg saved
col299_cam2_6.jpg saved
col316_cam1_3.jpg saved
col299_cam2_7.jpg saved
cam1 ----- (392, 8)
col316_cam1_4.jpg saved
col300_cam2_0.jpg saved
cam1 ----- (392, 7)
col316_cam1_5.jpg saved
col300_cam2_1.jpg saved
cam1 ----- (392, 8)
col316_cam1_6.jpg saved
col300_cam2_2.jpg saved
cam1 ----- (392, 8)
col316_cam1_7.jpg saved
cam1 ----- (392, 8)
col317_cam1_0.jpg saved
co

col312_cam2_6.jpg saved
cam1 ----- (212, 132)
col331_cam1_0.jpg saved
cam2 ----- (241, 119)
col312_cam2_7.jpg saved
cam1 ----- (219, 238)
col331_cam1_1.jpg saved
cam2 ----- (253, 341)
col313_cam2_0.jpg saved
cam1 ----- (226, 337)
col331_cam1_2.jpg saved
cam2 ----- (253, 341)
col313_cam2_1.jpg saved
cam1 ----- (234, 429)
col331_cam1_3.jpg saved
cam2 ----- (259, 438)
col313_cam2_2.jpg saved
cam1 ----- (393, 9)
col331_cam1_4.jpg saved
col313_cam2_3.jpg saved
cam1 ----- (393, 9)
col331_cam1_5.jpg saved
col313_cam2_4.jpg saved
cam1 ----- (392, 8)
col331_cam1_6.jpg saved
col313_cam2_5.jpg saved
cam1 ----- (392, 7)
col331_cam1_7.jpg saved
cam1 ----- (393, 9)
col332_cam1_0.jpg saved
col313_cam2_6.jpg saved
cam1 ----- (393, 9)
col332_cam1_1.jpg saved
col313_cam2_7.jpg saved
cam1 ----- (394, 9)
col332_cam1_2.jpg saved
col314_cam2_0.jpg saved
cam1 ----- (394, 9)
col332_cam1_3.jpg saved
col314_cam2_1.jpg saved
cam1 ----- (394, 9)
col332_cam1_4.jpg saved
col314_cam2_2.jpg saved
cam1 ----- (393, 9)


col326_cam2_1.jpg saved
cam1 ----- (281, 3)
col345_cam1_2.jpg saved
cam2 ----- (297, 4)
col326_cam2_2.jpg saved
cam1 ----- (281, 3)
col345_cam1_3.jpg saved
cam2 ----- (297, 4)
col326_cam2_3.jpg saved
cam1 ----- (280, 3)
col345_cam1_4.jpg saved
cam2 ----- (296, 4)
col326_cam2_4.jpg saved
cam1 ----- (280, 3)
col345_cam1_5.jpg saved
cam2 ----- (296, 4)
col326_cam2_5.jpg saved
cam1 ----- (280, 3)
col345_cam1_6.jpg saved
cam2 ----- (296, 4)
col326_cam2_6.jpg saved
cam1 ----- (280, 3)
col345_cam1_7.jpg saved
cam2 ----- (296, 4)
col326_cam2_7.jpg saved
cam1 ----- (281, 3)
col346_cam1_0.jpg saved
cam2 ----- (296, 4)
col327_cam2_0.jpg saved
cam1 ----- (281, 3)
col346_cam1_1.jpg saved
cam2 ----- (296, 4)
col327_cam2_1.jpg saved
cam1 ----- (281, 3)
col346_cam1_2.jpg saved
cam2 ----- (296, 4)
col327_cam2_2.jpg saved
cam1 ----- (281, 3)
col346_cam1_3.jpg saved
cam2 ----- (296, 4)
col327_cam2_3.jpg saved
cam1 ----- (281, 3)
col346_cam1_4.jpg saved
cam2 ----- (295, 4)
col327_cam2_4.jpg saved
cam1 ---

In [ ]:
cam0.join()
cam1.join()
cam0.close()  
cam1.close()

cv2.destroyAllWindows()

In [ ]:
while True:
    frame0 = cam0.current_frame
    frame1 = cam1.current_frame

    if frame0 is not None or frame1 is not None:
        # cv2.imshow('Camera 0', frame0)
        # cv2.imshow('Camera 1', frame1)
        ROI0 = cv2.warpPerspective(frame0, matrix1, (w1, h1))
        ROI1 = cv2.warpPerspective(frame1, matrix2, (w2, h2))
#         cv2.imshow('ROI0', ROI0)
#         cv2.imshow('ROI1', ROI1)
        orange_mask0, orange_object0 = orange_in_tableROI(ROI0, 1)
        orange_mask1, orange_object1 = orange_in_tableROI(ROI1, 2)
        cv2.imshow(f'ROI0 - orange', orange_object0)
        cv2.imshow(f'ROI1 - orange', orange_object1)
        is_orange_in_range = check_orange_in_range(orange_mask0, 0, 160)
        # is_orange_in_range = check_orange_in_range(orange_mask1, 0, 160)
        if is_orange_in_range: 
            save_frames = True
        if save_frames:
            ballPos0 = ballPosition_in_tableROI(ROI0, orange_mask0, orange_object0, 1)
            ballPos1 = ballPosition_in_tableROI(ROI1, orange_mask1, orange_object1, 2)
            cam1_points.append(ballPos0)
            cam2_points.append(ballPos1)
#             cv2.imwrite(f'images/ballPosDataCollect/col{dataColumn_counter0}_cam1_{frame_counter0}.jpg', ROI0)
#             cv2.imwrite(f'images/ballPosDataCollect/col{dataColumn_counter1}_cam2_{frame_counter1}.jpg', ROI1)
#             print(f'col{dataColumn_counter0}_cam1_{frame_counter0}.jpg saved')
#             print(f'col{dataColumn_counter1}_cam2_{frame_counter1}.jpg saved')
            frame_counter0 += 1 
            frame_counter1 += 1 
            if frame_counter0 >= 8: 
                with open('dataset_ballPosXY.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['cam1']+[str(x) for x in cam1_points]) 
                save_frames = False 
                frame_counter0 = 0 
                cam1_points = []
                dataColumn_counter0 += 1 
            if frame_counter1 >= 8: 
                with open('dataset_ballPosXY.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['cam2']+[str(x) for x in cam2_points]) 
                save_frames = False 
                frame_counter1 = 0 
                cam2_points = []
                dataColumn_counter1 += 1 

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break

# cam0.join()
# cam1.join()
cam0.close()  
cam1.close()

cv2.destroyAllWindows()